## For efficient masking add to each training polygon the tile name of the sentinel-2 tiles  

The sentinel missions map each tile every 5 days, the coordinates of each tile are fixed as well as the names  
Each tile has the size of 110 $km^2$ x 110 $km^2$ with 10 km on each side as overlap to the enclosed tiles 

Download tiling grid polygons: https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/data-products

Download german border: http://www.diva-gis.org/gdata
 - Select Germany and Administrative areas
   - DEU_adm1.shp contains the borders as polygons from each state
   - DEU_adm2.shp contains the borders as polygons from each district

Task:
1. Open trn_polygons + file with coordinates of the german borders and clip them to the borders
2. Open file with tile names + coordinates and add centroid coordinates
3. Add tile name to each training polygon and drop unnecessary columns
4. Save training polygons with tile name as geojson in ../src/preprocess_data_to_disk/data/training_polygons_germany.geojson
5. Save each tile with centroid from training polygons to ../src/download_sentinel_data/data/tiles_germany.geojson to use it for downloading the data

In [1]:
import fiona
import geopandas as gpd

### 1. Open trn_polygons + file with coordinates of the german borders and clip them to the borders

In [4]:
trn_polygons_path = r'../raw_data/trn_polygons.geojson'
trn_polygons = gpd.read_file(trn_polygons_path)
trn_polygons.head(3)

,Country,Province,Project,WRI_ref,Polygon Source,Date,building,operator,generator_source,amenity,...,layer,name,barrier,addr_housenumber,office,power,area,osm_id,military,geometry
0,China,Anhui,Chaohu A,nan,Satellite shows,2017-10-02 10:16:12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((117.59245 31.31579, 117.59229 31.315..."
1,China,Anhui,Chaohu A,nan,Satellite shows,2017-10-02 10:16:12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((117.58517 31.31303, 117.58733 31.313..."
2,China,Anhui,Chaohu A,nan,Satellite shows,2017-10-02 10:16:12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((117.59129 31.31898, 117.59261 31.319..."


In [8]:
trn_polygons.shape

(36882, 37)

In [7]:
# read german border polygon to filter trn_polygons on it.
# use DEU_adm1.shp and filter it on a certain state, to get the tiles for just one or more states of germany (same with DEU_adm2 on district level)
germany_borders_path = r'../raw_data/DEU_adm/DEU_adm0.shp'
germany_borders = gpd.read_file(germany_borders_path)
germany_borders.head()

,ID_0,ISO,NAME_0,OBJECTID_1,ISO3,NAME_ENGLI,NAME_ISO,NAME_FAO,NAME_LOCAL,NAME_OBSOL,...,CARICOM,EU,CAN,ACP,Landlocked,AOSIS,SIDS,Islands,LDC,geometry
0,86,DEU,Germany,62,DEU,Germany,GERMANY,Germany,Deutschland,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((8.70837 47.71556, 8.70918 47.7..."


In [9]:
# get only polygons which covers germany
# Takes a while to run 
trn_polygons_germany = gpd.clip(trn_polygons, germany_borders)
trn_polygons_germany.shape

(24233, 37)

In [51]:
# drop duplicates which are caused by clipping
trn_polygons_germany.drop_duplicates(subset='geometry', inplace=True)
trn_polygons_germany.shape

(23222, 37)

### 2. Open file with tile names + coordinates and add centroid coordinates

In [34]:
# to read a kml file into a geopandas dataframe, we need to activate the KML/kml driver in fiona (which is the underlying engine in geopandas)
# https://gis.stackexchange.com/questions/114066/handling-kml-csv-with-geopandas-drivererror-unsupported-driver-ucsv

fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable kml support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default
tiles_grid = gpd.read_file(r'../raw_data/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml')

In [35]:
# Name is the name of the associated tile
tiles_grid.head()

,Name,Description,geometry
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...


In [29]:
tiles_grid.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [40]:
# add centroid (center point of the tile), to use as footprint when query sentinel data
# convert to wkt, else geopandas throw's an error when saving to file

# Note the geometry is reprojected to epsg:3035, to calculate the centroid, and then reprojected back to epsg:4326
# epsg:3035 is the projection for central europe: https://epsg.io/3035

# See also:
# https://stackoverflow.com/questions/63004400/getting-a-userwarning-when-calculating-centroid-of-a-geoseries/63038899#63038899
tiles_grid['centroid_of_tile'] = tiles_grid.to_crs(epsg=3035).centroid.to_crs(epsg=4326).apply(lambda x: x.wkt)
tiles_grid.head()

,Name,Description,geometry,centroid_of_tile
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,POINT (178.56602271617683 -72.67913474116756)
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (179.46116994924037 -83.35283236512397)
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (179.8741151454747 -82.4613875791395)
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (-179.8013143911279 -81.57086876411002)
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (-179.53946066450573 -80.6804934882931)


In [41]:
tiles_grid.rename(columns={'Name': 'tile_name'}, inplace=True)

In [42]:
tiles_grid.head()

,tile_name,Description,geometry,centroid_of_tile
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,POINT (178.56602271617683 -72.67913474116756)
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (179.46116994924037 -83.35283236512397)
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (179.8741151454747 -82.4613875791395)
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (-179.8013143911279 -81.57086876411002)
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,POINT (-179.53946066450573 -80.6804934882931)


In [45]:
tiles_grid.shape

(56686, 4)

### 3. Add tile name to each training polygon and drop unnecessary columns

In [46]:
# map the polygons (=solar plants) on the name of the associated tile where the polygon lies within
# so later we can look them up by just look for the tile name instead to filter the dataframe on the geometry again (to speed up the process)
trn_polygons_germany_with_tiles_name = gpd.sjoin(trn_polygons_germany, tiles_grid, how='left', predicate='intersects')

In [47]:
trn_polygons_germany_with_tiles_name.shape

(32953, 41)

In [49]:
# drop duplicates which are caused by sjoin
trn_polygons_germany_with_tiles_name.drop_duplicates(subset=['geometry'], inplace=True)

In [52]:
trn_polygons_germany_with_tiles_name.shape

(23222, 41)

In [53]:
trn_polygons_germany_with_tiles_name.columns

Index(['Country', 'Province', 'Project', 'WRI_ref', 'Polygon Source', 'Date',
       'building', 'operator', 'generator_source', 'amenity', 'landuse',
       'power_source', 'shop', 'sport', 'tourism', 'way_area', 'access',
       'construction', 'denomination', 'historic', 'leisure', 'man_made',
       'natural', 'ref', 'religion', 'surface', 'z_order', 'layer', 'name',
       'barrier', 'addr_housenumber', 'office', 'power', 'area', 'osm_id',
       'military', 'geometry', 'index_right', 'tile_name', 'Description',
       'centroid_of_tile'],
      dtype='object')

In [55]:
# select only needed columns 
selected_trn_polygons_germany_tile_names = gpd.GeoDataFrame(trn_polygons_germany_with_tiles_name[['osm_id','tile_name', 'geometry', 'centroid_of_tile']], crs=trn_polygons_germany_with_tiles_name.crs, geometry=trn_polygons_germany_with_tiles_name.geometry).reset_index(drop=True)
selected_trn_polygons_germany_tile_names.head()

,osm_id,tile_name,geometry,centroid_of_tile
0,483534385.0,33UVS,"POLYGON ((13.88598 50.89644, 13.88598 50.89642...",POINT (14.35719383285053 50.955770194506044)
1,359898843.0,33UVT,"POLYGON ((13.95746 51.53471, 13.95746 51.53468...",POINT (14.344460359916278 51.85505191805214)
2,359898992.0,33UVT,"POLYGON ((13.95746 51.53479, 13.95746 51.53477...",POINT (14.344460359916278 51.85505191805214)
3,359899222.0,33UVT,"POLYGON ((13.95746 51.53487, 13.95746 51.53484...",POINT (14.344460359916278 51.85505191805214)
4,359899000.0,33UVT,"POLYGON ((13.95745 51.53494, 13.95745 51.53491...",POINT (14.344460359916278 51.85505191805214)


### 4. Save training polygons with tile name as geojson in ../src/preprocess_data_to_disk/data/trn_polygons_germany_tile_names.geojson.geojson

In [58]:
# save it to geojson (needs more space on disk)
selected_trn_polygons_germany_tile_names.to_file('../src/preprocess_data_to_disk/data/trn_polygons_germany_tile_names.geojson', driver='GeoJSON')

In [1]:
# Alternative save it as .shp file (needs less space) but produces more files and error
# selected_trn_polygons_germany_tile_names.to_file('../src/preprocess_data_to_disk/data/trn_polygons_germany_tile_names.shp')

#### The saved file is used for masking the satellite images in the dataset

### 5. Save each tile with centroid from training polygons to ../src/download_sentinel_data/data/tiles_germany.geojson to use it for downloading the data

In [56]:
# check how many tiles are needed for the training on all polygons in germany
len(set(selected_trn_polygons_germany_tile_names.tile_name))

63

In [63]:
tiles_germany = selected_trn_polygons_germany_tile_names.drop_duplicates(subset=['tile_name'])
tiles_germany.shape

(63, 4)

In [64]:
# save it to geojson (needs more space on disk)
tiles_germany.to_file('../src/download_sentinel_data/data/tiles_germany.geojson', driver='GeoJSON')

#### The saved file is used to download the sentinel data over germany